In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime,timedelta

In [3]:
API_KEY='973ed7749164da8ee67a51bcd8b4728b'
BASE_URL='https://api.openweathermap.org/data/2.5/'

In [4]:
def get_current_weather(city):
    url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" 
    response=requests.get(url)
    data=response.json()
    return {
        'city':data['name'],
        'current_temp':round(data['main']['temp']),
        'feels_like':round(data['main']['feels_like']),
        'temp_min':round(data['main']['temp_min']),
        'temp_max':round(data['main']['temp_max']),
        'humidity':round(data['main']['humidity']),
        'description':data['weather'][0]['description'],
        'country':data['sys']['country'],
    }